In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np 
import torch_geometric.nn as pyg_nn 
import torch_geometric.utils as pyg_utils
from torch_geometric.data import Dataset, Data, DataLoader
import torch.optim as optim
import os.path as osp
import scipy.io as sio
from dataset import QUASARDataset
from dual_model import DualModel

In [8]:
dir = '/home/hank/Datasets/QUASAR/small'
dataset = QUASARDataset(dir,num_graphs=100,remove_self_loops=True)
test_dir = '/home/hank/Datasets/QUASAR/small-test'
testset = QUASARDataset(test_dir,num_graphs=100,remove_self_loops=True)

Data graph type: 1.
Data graph type: 1.


In [9]:
GNN_TYPE = 'SAGE'
GNN_HIDDEN_DIM = 64
GNN_OUT_DIM = 64
GNN_LAYER = 4
NODE_MODE = 1
DATA_GRAPH_TYPE = 1
DROPOUT = 0
MLP_LAYER = 2
model   = DualModel(node_feature_mode=NODE_MODE,
                     gnn_type=GNN_TYPE,
                     mp_hidden_dim=GNN_HIDDEN_DIM,mp_output_dim=GNN_OUT_DIM,mp_num_layers=GNN_LAYER, 
                     dual_node_mlp_hidden_dim=GNN_HIDDEN_DIM,dual_node_mlp_output_dim=10,
                     node_mlp_num_layers=MLP_LAYER,
                     dual_edge_mlp_hidden_dim=GNN_HIDDEN_DIM,dual_edge_mlp_output_dim=6,
                     edge_mlp_num_layers=MLP_LAYER, 
                     dropout_rate=DROPOUT,
                     relu_slope=0.1)
model.load_state_dict(torch.load('./models/dual_model_SAGE_4_64_64_1_1_2000_0.0_2.pth'))
model.double()
model.eval()

Model: node_feature_mode = 1, mp_input_dim = 6, relu_slope = 0.1. GNN type: SAGE.


DualModel(
  (mp_convs): ModuleList(
    (0): SAGEConv(6, 64)
    (1): SAGEConv(64, 64)
    (2): SAGEConv(64, 64)
    (3): SAGEConv(64, 64)
    (4): SAGEConv(64, 64)
    (5): SAGEConv(64, 64)
  )
  (dual_node_mlp): ModuleList(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Linear(in_features=64, out_features=10, bias=True)
  )
  (dual_edge_mlp): ModuleList(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Linear(in_features=64, out_features=6, bias=True)
  )
)

In [10]:
# results on train set
loader = DataLoader(dataset,batch_size=1,shuffle=True)
train_loss = []
for batch in loader:
    _, S, Aty = model(batch)
    dual_loss = model.loss(batch,S,Aty)
    train_loss.append(dual_loss.item())
train_acc = torch.mean(torch.tensor(train_loss))
print('Train acc: {:.4f}.'.format(train_acc))

/home/hank/miniconda3/envs/gnnsdp/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Train acc: 0.0686.


In [11]:
# results on test set
loader = DataLoader(testset,batch_size=1,shuffle=True)
test_loss = []
for batch in loader:
    _, S, Aty = model(batch)
    dual_loss = model.loss(batch,S,Aty)
    test_loss.append(dual_loss.item())
test_acc = torch.mean(torch.tensor(test_loss))
print('Test acc: {:.4f}.'.format(test_acc))
    

Test acc: 0.2657.


In [12]:
# results of randmodel
randmodel = DualModel(node_feature_mode=NODE_MODE,
                     gnn_type=GNN_TYPE,
                     mp_hidden_dim=GNN_HIDDEN_DIM,mp_output_dim=GNN_OUT_DIM,mp_num_layers=GNN_LAYER, 
                     dual_node_mlp_hidden_dim=GNN_HIDDEN_DIM,dual_node_mlp_output_dim=10,
                     node_mlp_num_layers=MLP_LAYER,
                     dual_edge_mlp_hidden_dim=GNN_HIDDEN_DIM,dual_edge_mlp_output_dim=6,
                     edge_mlp_num_layers=MLP_LAYER, 
                     dropout_rate=DROPOUT,
                     relu_slope=0.1)
randmodel.double()
randmodel.eval()

loader = DataLoader(dataset,batch_size=1,shuffle=True)
train_loss = []
for batch in loader:
    _, S, Aty = randmodel(batch)
    dual_loss = randmodel.loss(batch,S,Aty)
    train_loss.append(dual_loss.item())
train_acc = torch.mean(torch.tensor(train_loss))
print('Rand train acc: {:.4f}.'.format(train_acc))

loader = DataLoader(testset,batch_size=1,shuffle=True)
test_loss = []
for batch in loader:
    _, S, Aty = randmodel(batch)
    dual_loss = randmodel.loss(batch,S,Aty)
    test_loss.append(dual_loss.item())
test_acc = torch.mean(torch.tensor(test_loss))
print('Rand test acc: {:.4f}.'.format(test_acc))

Model: node_feature_mode = 1, mp_input_dim = 6, relu_slope = 0.1. GNN type: SAGE.
Rand train acc: 1.0000.
Rand test acc: 1.0000.
